In [1]:
import shap
print(shap.__version__)

/opt/miniconda3/envs/rankingSHAP/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


0.45.1


In [2]:
# import pandas as pd
# import random

# topics = [
#     "weather", "sports", "finance", "technology", "health",
#     "music", "education", "travel", "food", "movies"
# ]

# doc_templates = [
#     "This document discusses {} in detail.",
#     "An overview of the latest trends in {}.",
#     "The impact of {} on daily life.",
#     "Recent developments in {}.",
#     "Understanding the basics of {}."
# ]

# query_doc_pairs = []
# for i in range(10):
#     topic = random.choice(topics)
#     query = f"What is new in {topic}?"
#     doc = random.choice(doc_templates).format(topic)
#     query_doc_pairs.append([query, doc])

# df = pd.DataFrame(query_doc_pairs, columns=["query", "document"])
# print(df)


In [3]:
from elasticsearch import Elasticsearch, helpers
import time

# 1. Connect to ES (edit the host/port as needed)
es = Elasticsearch("http://localhost:9200")
index_name = "bm25-demo"

# 2. Delete index if it exists (for repeatable demo)
if es.indices.exists(index=index_name):
    es.indices.delete(index=index_name)

# 3. Create index with BM25 (standard analyzer by default)
mapping = {

    "mappings": {
        "properties": {
            "content": {
                "type": "text",  # Uses BM25 and standard analyzer by default
            }
        }
    }
}
es.indices.create(index=index_name, body=mapping)
print(f"Created index '{index_name}'.")

# 4. Index a few documents
docs = [
    {"content": "The stock market saw gains today in the finance sector."},
    {"content": "Technology advances are happening at a rapid pace."},
    {"content": "Healthy eating and exercise improve overall health."},
    {"content": "Weather forecasts predict rain this week."},
    {"content": "Financial experts recommend saving early for retirement."},
]
actions = [{"_index": index_name, "_source": doc} for doc in docs]
helpers.bulk(es, actions)
print("Indexed documents.")
time.sleep(1)  # Let ES index the docs



import sys
sys.path.append('/Users/keqiaoli/Desktop/RankingSHAP/RankingShap/')
# import rankingSHAP_test_class_cutomized
from rankingSHAP_test_class_cutomized import extract_es_features_optimized 
# from rankingSHAP_test_class import extract_es_features_optimized
# ----- Usage -----
es = Elasticsearch("http://localhost:9200")
index_name = "bm25-demo"

# Extract features using the optimized method
features = extract_es_features_optimized(es, index_name, field="content")
print(features)








Created index 'bm25-demo'.
Indexed documents.
['a', 'advances', 'and', 'are', 'at', 'early', 'eating', 'exercise', 'experts', 'finance', 'financial', 'for', 'forecasts', 'gains', 'happening', 'health', 'healthy', 'improve', 'in', 'market', 'overall', 'pace', 'predict', 'rain', 'rapid', 'recommend', 'retirement', 'saving', 'saw', 'sector', 'stock', 'technology', 'the', 'this', 'today', 'weather', 'week']


In [4]:

from elasticsearch import Elasticsearch, helpers
import sys
import os
from pathlib import Path
import time

def extract_features_from_documents(documents, index_name="bm25-demo", es_host="http://localhost:9200", field="content"):
    """
    Extract features from documents using Elasticsearch and RankingSHAP.
    
    Args:
        documents (list): List of dictionaries with document content
                         Format: [{"content": "text1"}, {"content": "text2"}, ...]
        index_name (str): Name of the Elasticsearch index to create
        es_host (str): Elasticsearch host URL
        field (str): Field name to analyze for features
    
    Returns:
        features: Extracted features from RankingSHAP, or None if error occurs
    """
    
    # 1. Connect to ES with error handling
    try:
        es = Elasticsearch(es_host)
        if not es.ping():
            raise ConnectionError("Cannot connect to Elasticsearch")
        print("Successfully connected to Elasticsearch")
    except Exception as e:
        print(f"ES connection failed: {e}")
        return None
    
    # 2. Delete index if it exists (for repeatable execution)
    try:
        if es.indices.exists(index=index_name):
            es.indices.delete(index=index_name)
            print(f"Deleted existing index '{index_name}'")
    except Exception as e:
        print(f"Error deleting index: {e}")
        return None
    
    # 3. Create index with BM25 (standard analyzer by default)
    mapping = {
        "mappings": {
            "properties": {
                field: {
                    "type": "text",  # Uses BM25 and standard analyzer by default
                }
            }
        }
    }
    
    try:
        es.indices.create(index=index_name, **mapping)
        print(f"Created index '{index_name}'.")
    except Exception as e:
        print(f"Error creating index: {e}")
        return None
    
    # 4. Index the provided documents
    if not documents:
        print("No documents provided")
        return None
    
    actions = [{"_index": index_name, "_source": doc} for doc in documents]
    
    try:
        response = helpers.bulk(es, actions)
        print(f"Successfully indexed {len(documents)} documents")
    except helpers.BulkIndexError as e:
        print(f"Bulk indexing errors: {e.errors}")
        return None
    except Exception as e:
        print(f"Unexpected error during bulk indexing: {e}")
        return None

    # # Refresh index to make documents searchable immediately
    try:
        es.indices.refresh(index=index_name)
        print("Index refreshed")
    except Exception as e:
        print(f"Error refreshing index: {e}")
    
    # 5. Import custom module and extract features
    try:
        # Get the current script's directory
        current_dir = Path.cwd()
        rankingshap_path = current_dir / "RankingSHAP" / "RankingShap"
        
        # Add to path if it exists
        if rankingshap_path.exists():
            sys.path.insert(0, str(rankingshap_path))
        else:
            # Fallback to your original path (make it configurable)
            fallback_path = "/Users/keqiaoli/Desktop/RankingSHAP/RankingShap/"
            if os.path.exists(fallback_path):
                sys.path.append(fallback_path)
            else:
                raise ImportError(f"RankingSHAP module not found at expected locations")
        
        from rankingSHAP_test_class_cutomized import extract_es_features_optimized
        
        # Extract features using the optimized method
        features = extract_es_features_optimized(es, index_name, field=field)
        print("Extracted features:")
        print(features)
        return features
        
    except ImportError as e:
        print(f"Import error: {e}")
        print("Please check the path to your RankingSHAP module")
        return None
    except Exception as e:
        print(f"Error extracting features: {e}")
        return None



In [5]:

"""Example of how to use the extract_features_from_documents function"""

# Sample documents
docs = [
    {"content": "The stock market saw gains today in the finance sector."},
    {"content": "Technology advances are happening at a rapid pace."},
    {"content": "Healthy eating and exercise improve overall health."},
    {"content": "Weather forecasts predict rain this week."},
    {"content": "Financial experts recommend saving early for retirement."},
]

# Extract features
features = extract_features_from_documents(documents=docs)

if features is not None:
    print("Feature extraction completed successfully!")

else:
    print("Feature extraction failed!")


Successfully connected to Elasticsearch
Deleted existing index 'bm25-demo'
Created index 'bm25-demo'.
Successfully indexed 5 documents
Index refreshed
Extracted features:
['a', 'advances', 'and', 'are', 'at', 'early', 'eating', 'exercise', 'experts', 'finance', 'financial', 'for', 'forecasts', 'gains', 'happening', 'health', 'healthy', 'improve', 'in', 'market', 'overall', 'pace', 'predict', 'rain', 'rapid', 'recommend', 'retirement', 'saving', 'saw', 'sector', 'stock', 'technology', 'the', 'this', 'today', 'weather', 'week']
Feature extraction completed successfully!


In [11]:
from elasticsearch import Elasticsearch, helpers
from typing import List, Dict, Any, Optional
import uuid

class ElasticsearchSparseSearchModel:
    """
    A simple Elasticsearch sparse search model with scikit-learn-like interface.
    Uses BM25 scoring for sparse retrieval.
    """
    
    def __init__(self, es_host: str = "http://localhost:9200", index_name: Optional[str] = None):
        """
        Initialize the Elasticsearch sparse search model.
        
        Args:
            es_host (str): Elasticsearch host URL
            index_name (str, optional): Index name. If None, generates a random one.
        """
        self.es_host = es_host
        self.index_name = index_name or f"sparse_search_{uuid.uuid4().hex[:8]}"
        self.es = None
        self.is_fitted = False
        self.field_name = "content"
        
    def _connect(self):
        """Establish connection to Elasticsearch."""
        if self.es is None:
            try:
                self.es = Elasticsearch(self.es_host)
                if not self.es.ping():
                    raise ConnectionError("Cannot connect to Elasticsearch")
                print(f"Connected to Elasticsearch at {self.es_host}")
            except Exception as e:
                raise ConnectionError(f"Failed to connect to Elasticsearch: {e}")
    
    def fit(self, documents: List[str]) -> 'ElasticsearchSparseSearchModel':
        """
        Fit the model by indexing documents into Elasticsearch.
        
        Args:
            documents (List[str]): List of document texts to index
            
        Returns:
            self: Returns the instance for method chaining
        """
        self._connect()
        
        # Delete existing index if it exists
        if self.es.indices.exists(index=self.index_name):
            self.es.indices.delete(index=self.index_name)
            print(f"Deleted existing index '{self.index_name}'")
        
        # Create index with BM25 settings
        mapping = {
            "mappings": {
                "properties": {
                    self.field_name: {
                        "type": "text",
                        "analyzer": "standard"  # Uses BM25 by default
                    }
                }
            }
        }
        
        try:
            self.es.indices.create(index=self.index_name, **mapping)
            print(f"Created index '{self.index_name}'")
        except Exception as e:
            raise RuntimeError(f"Failed to create index: {e}")
        
        # Index documents
        actions = []
        for i, doc in enumerate(documents):
            actions.append({
                "_index": self.index_name,
                "_id": i,
                "_source": {self.field_name: doc}
            })
        
        try:
            helpers.bulk(self.es, actions)
            self.es.indices.refresh(index=self.index_name)
            print(f"Indexed {len(documents)} documents")
            self.is_fitted = True
        except Exception as e:
            raise RuntimeError(f"Failed to index documents: {e}")
        
        return self
    
    def predict(self, queries: List[str], top_k: int = 5) -> List[List[Dict[str, Any]]]:
        """
        Perform sparse search predictions for given queries.
        
        Args:
            queries (List[str]): List of query strings
            top_k (int): Number of top results to return per query
            
        Returns:
            List[List[Dict]]: For each query, returns list of top_k results.
                            Each result contains: {'doc_id', 'score', 'content'}
        """
        if not self.is_fitted:
            raise RuntimeError("Model must be fitted before making predictions. Call .fit() first.")
        
        results = []
        
        for query in queries:
            # Elasticsearch query using BM25
            search_body = {
                "query": {
                    "match": {
                        self.field_name: query
                    }
                },
                "size": top_k,
                "_source": [self.field_name]
            }
            
            try:
                response = self.es.search(index=self.index_name, body=search_body)
                
                query_results = []
                for hit in response['hits']['hits']:
                    result = {
                        'doc_id': hit['_id'],
                        'score': hit['_score'],
                        'content': hit['_source'][self.field_name]
                    }
                    query_results.append(result)
                
                results.append(query_results)
                
            except Exception as e:
                print(f"Error searching for query '{query}': {e}")
                results.append([])  # Return empty results on error
        
        return results
    
    def predict_single(self, query: str, top_k: int = 5) -> List[Dict[str, Any]]:
        """
        Convenience method for single query prediction.
        
        Args:
            query (str): Single query string
            top_k (int): Number of top results to return
            
        Returns:
            List[Dict]: List of top_k results for the query
        """
        results = self.predict([query], top_k)
        return results[0] if results else []
    
    def cleanup(self):
        """Delete the index and clean up resources."""
        if self.es and self.es.indices.exists(index=self.index_name):
            self.es.indices.delete(index=self.index_name)
            print(f"Deleted index '{self.index_name}'")



# Sample documents
documents = [
    "The stock market saw gains today in the finance sector.",
    "Technology advances are happening at a rapid pace.",
    "Healthy eating and exercise improve overall health.",
    "Weather forecasts predict rain this week.",
    "Financial experts recommend saving early for retirement.",
    "Machine learning models require large datasets for training.",
    "The weather today is sunny and warm.",
    "Exercise and diet are key to maintaining good health."
]

# Create and fit the model
model = ElasticsearchSparseSearchModel()
# Fit the model
print("Fitting model with documents...")
model.fit(documents)




Fitting model with documents...
Connected to Elasticsearch at http://localhost:9200
Created index 'sparse_search_a9557d37'
Indexed 8 documents


In [12]:
from utils.get_explanations import calculate_all_query_explanations
from utils.helper_functions import get_data
import lightgbm
import numpy as np
from scipy.stats import kendalltau
from utils.background_data import BackgroundData
from approaches.ranking_shap import RankingShap
from approaches.ranking_lime import RankingLIME
from approaches.greedy_listwise import GreedyListwise
from approaches.pointwise_lime import AggregatedLime
from approaches.pointwise_shap import AggregatedShap
from approaches.random_explainer import RandomExplainer
from pathlib import Path




explanation_size = 5


num_features = len(features)
# background_data = BackgroundData(
#     np.full((1, num_features), '<unk>'),  # Replace with the desired number of rows and features
#     summarization_type=None,
# )

background_data_new = np.full((1, num_features), '<unk>', dtype=object)







In [13]:
background_data_new

array([['<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>',
        '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>',
        '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>',
        '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>',
        '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>',
        '<unk>', '<unk>']], dtype=object)

In [14]:
# import approaches.ranking_shap as ranking_shap_module
# import importlib
# importlib.reload(ranking_shap_module)

# from approaches.ranking_shap import RankingShap




# rank_similarity_coefficient = lambda x, y: kendalltau(x, y)[0]

# # Define all the explainers
# ranking_shap_explainer = RankingShap(
#     permutation_sampler="kernel",
#     background_data=background_data_new,
#     original_model=model.predict,
#     explanation_size=3,
#     name="rankingshap",
#     rank_similarity_coefficient=rank_similarity_coefficient,
# )

# features_selection, feature_attribution = ranking_shap_explainer.get_query_explanation(
#     query_features=features, query_id="12345"
# )


In [29]:
documents

['The stock market saw gains today in the finance sector.',
 'Technology advances are happening at a rapid pace.',
 'Healthy eating and exercise improve overall health.',
 'Weather forecasts predict rain this week.',
 'Financial experts recommend saving early for retirement.',
 'Machine learning models require large datasets for training.',
 'The weather today is sunny and warm.',
 'Exercise and diet are key to maintaining good health.']

In [53]:
import sparse_search_bm25 as sparse_search_bm25
import importlib
importlib.reload(sparse_search_bm25)
import json

from sparse_search_bm25 import BM25SearchEngine, demo_bm25_search

query = "What is new in stock market?"
preds = demo_bm25_search(query,
                 documents,
                 top_k=3)

Indexed 3 documents

Query: 'What is new in stock market?'

1. Score: 1.4166

2. Score: 0.0000

3. Score: 0.0000


In [58]:
def rank_list(preds):
    """
    returns ndarray containing rank(i) for documents at position i
    """
    vector = np.array([pred['_score'] for pred in preds])
    temp = vector.argsort()[::-1]
    ranks = np.empty_like(temp)
    ranks[temp] = np.arange(1, len(vector) + 1)

    return ranks

In [59]:
rank_list(preds)

array([1, 3, 2])

In [ ]:
import approaches.ranking_shap as ranking_shap_module
import importlib
importlib.reload(ranking_shap_module)

from approaches.ranking_shap import RankingShap
import rbo
safe_attributions_to = '/Users/keqiaoli/Desktop/RankingSHAP/RankingShap/test_results.csv'


rank_similarity_coefficient = lambda x, y: rbo.RankingSimilarity(x, y)

# Define all the explainers
ranking_shap_explainer = RankingShap(
    permutation_sampler="kernel",
    background_data=background_data_new,
    original_model=demo_bm25_search,
    explanation_size=20,
    name="rankingshap",
    rank_similarity_coefficient=rank_similarity_coefficient,
)

feature_attribution = ranking_shap_explainer.get_query_explanation(
    query_features=features, documents = documents, query = query
)

feature_attribution.safe_to_file(safe_attributions_to)
# features_selection, feature_attribution = ranking_shap_explainer.get_query_explanation(
#     query_features=features, documents = documents, query_id=""
# )

# all_masks = ranking_shap_explainer.all_masks
# print(np.array(all_masks).shape)
# print(all_masks)


  0%|          | 0/1 [00:00<?, ?it/s]

array================>: [['<keep>' '<keep>' '<keep>' '<keep>' '<keep>' '<keep>' '<keep>' '<keep>'
  '<keep>' '<keep>' '<keep>' '<keep>' '<keep>' '<keep>' '<keep>' '<keep>'
  '<keep>' '<keep>' '<keep>' '<keep>' '<keep>' '<keep>' '<keep>' '<keep>'
  '<keep>' '<keep>' '<keep>' '<keep>' '<keep>' '<keep>' '<keep>' '<keep>'
  '<keep>' '<keep>' '<keep>' '<keep>' '<keep>']]
(1, 37)
==========> code goes here !!!!!
Query: What is new in stock market?
<function demo_bm25_search at 0x1780e2950>
Indexed 3 documents

Query: 'What is new in stock market?'

1. Score: 1.4166

2. Score: 0.0000

3. Score: 0.0000
Original rank: [1 3 2]

Mask 0: replaced_with_unk = set()
Indexed 3 documents

Query: 'What is new in stock market?'

1. Score: 1.4166

2. Score: 0.0000

3. Score: 0.0000
New rank for mask 0: [1 3 2]
[1.0]
array================>: [['<keep>' '<unk>' '<unk>' ... '<unk>' '<unk>' '<unk>']
 ['<unk>' '<keep>' '<keep>' ... '<keep>' '<keep>' '<keep>']
 ['<unk>' '<keep>' '<unk>' ... '<unk>' '<unk>' '<unk

100%|██████████| 1/1 [00:00<00:00,  2.53it/s]


1. Score: 0.4722

2. Score: 0.0000

3. Score: 0.0000
New rank for mask 554: [1 3 2]

Mask 555: replaced_with_unk = {'are', 'eating', 'for', 'sector', 'the', 'this', 'experts', 'stock', 'rapid'}
Indexed 3 documents

Query: 'What is new in stock market?'

1. Score: 0.9444

2. Score: 0.0000

3. Score: 0.0000
New rank for mask 555: [1 3 2]

Mask 556: replaced_with_unk = {'eating', 'for', 'and', 'sector', 'finance', 'market', 'predict', 'stock', 'the', 'saw', 'this', 'today', 'experts', 'overall', 'gains', 'happening', 'recommend', 'week', 'rain', 'saving', 'early', 'at', 'technology', 'retirement', 'pace', 'forecasts', 'weather', 'a', 'financial', 'rapid', 'improve', 'advances', 'in'}
Indexed 3 documents

Query: 'What is new in stock market?'

1. Score: 0.0000

2. Score: 0.0000

3. Score: 0.0000
New rank for mask 556: [3 2 1]

Mask 557: replaced_with_unk = {'are', 'healthy', 'health', 'exercise'}
Indexed 3 documents

Query: 'What is new in stock market?'

1. Score: 1.4166

2. Score: 0.000

TypeError: cannot unpack non-iterable AttributionExplanation object

In [75]:
features

['a',
 'advances',
 'and',
 'are',
 'at',
 'early',
 'eating',
 'exercise',
 'experts',
 'finance',
 'financial',
 'for',
 'forecasts',
 'gains',
 'happening',
 'health',
 'healthy',
 'improve',
 'in',
 'market',
 'overall',
 'pace',
 'predict',
 'rain',
 'rapid',
 'recommend',
 'retirement',
 'saving',
 'saw',
 'sector',
 'stock',
 'technology',
 'the',
 'this',
 'today',
 'weather',
 'week']

In [30]:
# import re

# def replace_words_in_sentences(documents, words_to_replace, unk_token="<unk>", case_sensitive=False):
#     if not words_to_replace:
#         return documents
    
#     # Sort by length to match longer words first (avoids subword matching confusion)
#     sorted_words = sorted(words_to_replace, key=len, reverse=True)
#     # Escape special characters
#     escaped_words = [re.escape(w) for w in sorted_words]
#     # Build regex pattern: \b matches word boundaries (including punctuation)
#     pattern_str = r'\b(' + '|'.join(escaped_words) + r')\b'
#     flags = 0 if case_sensitive else re.IGNORECASE
#     pattern = re.compile(pattern_str, flags)
#     # Replace, preserving surrounding punctuation
#     return [pattern.sub(unk_token, sent) for sent in documents]

# # Example setup
# query_features = ["a", "red", "cat", "stock", "today"]
# array = [
#     ["<keep>", "<unk>", "<keep>", "<keep>", "<unk>"],
#     ["<unk>", "<unk>", "<keep>", "<unk>", "<keep>"]
# ]
# documents = [
#     "today, the RED| cat jumps over the stock",
#     "a red cat in the stock market",
#     "today is a good day for the cat"
# ]

# # For each mask
# for mask_idx, mask in enumerate(array):
#     replaced_with_unk = set(q for q, a in zip(query_features, mask) if a == "<unk>")
#     print(f"\nMask {mask_idx}: replaced_with_unk = {replaced_with_unk}")
#     new_docs = replace_words_in_sentences(documents, replaced_with_unk)
#     for sent in new_docs:
#         print(sent)


In [32]:
documents

['The stock market saw gains today in the finance sector.',
 'Technology advances are happening at a rapid pace.',
 'Healthy eating and exercise improve overall health.',
 'Weather forecasts predict rain this week.',
 'Financial experts recommend saving early for retirement.',
 'Machine learning models require large datasets for training.',
 'The weather today is sunny and warm.',
 'Exercise and diet are key to maintaining good health.']

In [37]:
docuemnts

NameError: name 'docuemnts' is not defined

In [42]:
background_data_new

array([['<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>',
        '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>',
        '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>',
        '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>',
        '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>',
        '<unk>', '<unk>']], dtype=object)

In [44]:
import approaches.ranking_shap as ranking_shap_module
import importlib
importlib.reload(ranking_shap_module)

from approaches.ranking_shap import RankingShap




rank_similarity_coefficient = lambda x, y: kendalltau(x, y)[0]

# Define all the explainers
ranking_shap_explainer = RankingShap(
    permutation_sampler="kernel",
    background_data=background_data_new,
    original_model=demo_bm25_search,
    explanation_size=20,
    name="rankingshap",
    rank_similarity_coefficient=rank_similarity_coefficient,
)

features_selection, feature_attribution = ranking_shap_explainer.get_query_explanation(
    query_features=features, documents = documents, query_id=""
)


100%|██████████| 1/1 [00:00<00:00, 28.90it/s]

array================>: [['<keep>' '<keep>' '<keep>' '<keep>' '<keep>' '<keep>' '<keep>' '<keep>'
  '<keep>' '<keep>' '<keep>' '<keep>' '<keep>' '<keep>' '<keep>' '<keep>'
  '<keep>' '<keep>' '<keep>' '<keep>' '<keep>' '<keep>' '<keep>' '<keep>'
  '<keep>' '<keep>' '<keep>' '<keep>' '<keep>' '<keep>' '<keep>' '<keep>'
  '<keep>' '<keep>' '<keep>' '<keep>' '<keep>']]
(1, 37)
==========> code goes here !!!!!
['a', 'advances', 'and', 'are', 'at', 'early', 'eating', 'exercise', 'experts', 'finance', 'financial', 'for', 'forecasts', 'gains', 'happening', 'health', 'healthy', 'improve', 'in', 'market', 'overall', 'pace', 'predict', 'rain', 'rapid', 'recommend', 'retirement', 'saving', 'saw', 'sector', 'stock', 'technology', 'the', 'this', 'today', 'weather', 'week']

Mask 0: replaced_with_unk = set()
The stock market saw gains today in the finance sector.
Technology advances are happening at a rapid pace.
Healthy eating and exercise improve overall health.
array================>: [['<unk>' '

TypeError: cannot unpack non-iterable NoneType object